# CodeBert Grid Experiment Evaluation

Nice to see you around! Have a seat.
Would you like a drink? Maybe a cigar?

Make sure to have all required dependencies installed - they are listed in the [environment.yml](./environment.yml). 
You create a conda environment from the yml using 

```
conda env create -f environment.yml
conda activate Lampion-Codebert-Evaluation
```

Make sure to run your Jupyter Notebook from that environment! 
Otherwise you are (still) missing the dependencies. 

**OPTIONALLY** you can use the environment in which your jupter notebook is already running, with starting a new terminal (from jupyter) and run 

```
conda env update --prefix ./env --file environment.yml  --prune
```

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt

import nltk
nltk.download("punkt")
# Homebrew Imports (python-file next to this)
import bleu_evaluator as foreign_bleu

## Data-Loading / Preparation

Make sure that your dataset looks like described in the [Readme](./README.md), that is 

```
./data
    /GridExp_XY
        /configs
            /reference
                test_0.gold
                test_0.output
                bleu.txt (optional, can be created below)
            /config_0
                config.properties
                test_0.gold
                test_0.output
                bleu.txt (optional, can be created below)
            /config_1
                config.properties
                test_0.gold
                test_0.output
                bleu.txt (optional, can be created below)
    ...
```

where the configs **must** be numbered to be correctly detected. 

In [ ]:
# This runs the bleu-score upon the config files, creating the bleu.txt's 
# If your data package was provided including the txt you dont need to do this. 
# Existing bleu.txt's will be overwritten. 

#!./metric_runner.sh ./data/PreliminaryResults/

In [ ]:
data_directory = "./data/PreliminaryResults"

# These archetypes are later used to group the configurations
# While to grouping is up to you, right here it simply is one archetype for each transformation type, 
# Grouping together different configs with the same transformations applied (but different #Transformations)
config_archetypes = {
    "config_0":"if-true","config_1":"if-true","config_2":"if-true",
    "config_3":"mixed names(pseudo)","config_4":"mixed names(pseudo)","config_5":"mixed names(pseudo)",
    "config_6":"add-neutral","config_7":"add-neutral","config_8":"add-neutral",
    "config_9":"mixed-names(random)","config_10":"mixed-names(random)","config_11":"mixed-names(random)"
}

print(f"looking for results in {data_directory}" )

results={}

for root,dirs,files in os.walk(data_directory):
    for name in files:
        if ".gold" in name:
            directory = os.path.basename(root)
            results[directory]={}
            
            results[directory]["result_file"]=os.path.join(root,"test_0.output")
            results[directory]["gold_file"]=os.path.join(root,"test_0.gold")
            results[directory]["bleu_file"]=os.path.join(root,"bleu.txt")
            if os.path.exists(os.path.join(root,"config.properties")):
                results[directory]["property_file"]=os.path.join(root,"config.properties")

In [ ]:
def load_properties(filepath, sep='=', comment_char='#'):
    """
    Read the file passed as parameter as a properties file.
    """
    props = {}
    with open(filepath, "rt") as f:
        for line in f:
            l = line.strip()
            if l and not l.startswith(comment_char):
                key_value = l.split(sep)
                key = key_value[0].strip()
                value = sep.join(key_value[1:]).strip().strip('"') 
                props[key] = value 
    return props

print("reading in property-files")

for key in results.keys():
    if "property_file" in results[key].keys():
        results[f"{key}"]["properties"]=load_properties(results[key]["property_file"])

print("done reading the properties")

In [ ]:
print("reading in result-files")

for key in results.keys():
    result_file = results[key]["result_file"]
    f = open(result_file)
    lines=f.readlines()
    results[key]["results"]={}
    for l in lines:
        num = int(l.split("\t")[0])
        content = l.split("\t")[1]
        content = content.strip()
        results[key]["results"][num] = content
    f.close()
    
    gold_file = results[key]['gold_file']
    gf = open(gold_file)
    glines=gf.readlines()
    results[key]["gold_results"]={}
    for gl in glines:
        num = int(gl.split("\t")[0])
        content = gl.split("\t")[1]
        content = content.strip()
        results[key]["gold_results"][num] = content
    gf.close()

print("done reading the result files")
# Comment this in for inspection of results
#results

In [ ]:
print("reading in the bleu-scores")

for key in results.keys():
    bleu_file = results[key]["bleu_file"]
    f = open(bleu_file)
    score=f.readlines()[0]
    results[key]["bleu"]=float(score)
    f.close()
    
print("done reading the bleu-scores")

#results["config_0"]["bleu"]

In [ ]:
for key in results.keys():
    if "property_file" in results[key].keys():
        results[key]["archetype"]=config_archetypes[key]

## Bleu-Scores

In the following, the BLEU-scores will be calculated using the foreign libary. 
While there have been minor changes to standard-BLEU, it is the same as used in the original experiment.

The aggregated BLEU-Scores will be stored to the results.

In [ ]:
bleu_data = {}
archetypes = set([results[k]["archetype"] for k in results.keys() if "archetype" in results[k].keys()])
for archetype in archetypes:
    bleu_data[archetype]={}
    bleu_data[archetype][0]=results["reference"]["bleu"]
    relevant_configs = [k for k 
                        in results.keys() 
                        if "archetype" in results[k].keys() 
                        and results[k]["archetype"]==archetype]
    for c in relevant_configs:
        bleu_data[archetype][results[c]["properties"]["transformations"]]=results[c]["bleu"]
   
bleu_data_df = pd.DataFrame.from_dict(bleu_data)
bleu_data_df

In [ ]:
plt.ylabel("BLEU-Score")
plt.xlabel("# Transformations")

plot = plt.plot(bleu_data_df,marker="o")

plt.legend(bleu_data_df.columns)
plt.show()

## Samples

Before the samples can be inspected, the items need to be re-indexed. 
While all config_results are in the reference_results, there might is an issue with the data being shuffeld. 

To fix this, a reindexing is done.

In [ ]:
%%time
#Reindexing Pseudocode

def lookup_index(sentence, comparison_dict):
    for (gold_key,gold_value) in comparison_dict.items():
        if sentence == gold_value:
            return gold_key
    return -1

# For each config (that is not reference)
    # Create a lookup of reference_gold_index -> config_gold_index
    # Invert the lookup 
    # Make a new dictionary where
        # For every key of the config_gold
        # lookup the key of the reference_gold
        # And fill it with {reference_gold_key : config_gold_value}
        # Do the same with the non-gold results
        # Fill it with {reference_gold_key : config_result_value}
    # Set result[config_X]["gold_results"] to the newly created, matching index one 
    # same for non-gold-results
    
for config in [key for key in results.keys() if "reference" != key]:
    keyMapping={}
    for (k,v) in results[config]["gold_results"].items():
        gk = lookup_index(v,results["reference"]["gold_results"])
        keyMapping[k]=gk
    new_gold_results={}
    new_results={}
    for (config_key,gold_key) in keyMapping.items():
        if gold_key != -1:
            new_gold_results[gold_key]=results[config]["gold_results"][config_key]
            new_results[gold_key]=results[config]["results"][config_key]
    results[config]["gold_results"]=new_gold_results
    results[config]["results"]=new_results
    #print(config,keyMapping)

In [ ]:
sample_index = 250
print(results["reference"]["gold_results"][sample_index] )
#print(results["config_2"]["gold_results"][sample_index])
print()
print(results["reference"]["results"][sample_index])
print(results["config_2"]["results"][sample_index])

In [ ]:
def jaccard_wrapper(sentenceA,sentenceB,ngram=1):
    tokensA = nltk.word_tokenize(sentenceA)
    tokensB = nltk.word_tokenize(sentenceB)

    ngA_tokens = set(nltk.ngrams(tokensA, n=ngram))
    ngB_tokens = set(nltk.ngrams(tokensB, n=ngram))
    
    return nltk.jaccard_distance(ngA_tokens, ngB_tokens)

In [ ]:
def bleu_wrapper(sentence_to_check,reference):
    check_tokens = nltk.word_tokenize(sentence_to_check)
    ref_tokens = nltk.word_tokenize(reference)
    
    # From comparing the foreign_bleu and nltk the method4 seems to match
    # The Paper mentiones the BLEU-4-Score with a citation to chen & cherry
    # I wish I could be named chen & cherry, its a very cool name. 
    chencherry = nltk.translate.bleu_score.SmoothingFunction()
    smooth_fn = chencherry.method4
    
    return nltk.translate.bleu_score.sentence_bleu([ref_tokens],check_tokens,smoothing_function=smooth_fn)

bleu_wrapper(results["config_2"]["results"][sample_index],results["config_2"]["gold_results"][sample_index])

## Transformation Scores

from the paper, we provide two ways of checking on the transformation score: 

```
delta_tscore = bleu(gold,reference) - bleu(gold,altered)
```

and 

```
tquot = bleu(gold,altered) / bleu(gold,reference)
```

which we can perfectly write as python and see how they are doing. 


In [ ]:
delta_tscore = lambda gold,reference,altered : bleu_wrapper(reference,gold) - bleu_wrapper(altered,gold)

tquot = lambda gold,reference,altered : bleu_wrapper(altered,gold) / bleu_wrapper(reference,gold)

In [ ]:
delta_tscore(results["config_2"]["gold_results"][sample_index],results["reference"]["results"][sample_index],results["config_2"]["results"][sample_index])

In [ ]:
tquot(results["config_2"]["gold_results"][sample_index],results["reference"]["results"][sample_index],results["config_2"]["results"][sample_index])

**Hall of Shame** 

worst entries in terms of t-score (either one)

In [ ]:
%%time
n = 10
worst_n = []
best_n = []

for config in [key for key in results.keys() if "reference" != key]:
    running_agg = []
    for index in list(results[config]["results"].keys()):
        gold = results["reference"]["gold_results"][index]
        reference = results["reference"]["results"][index]
        altered = results[config]["results"][index]
        try:
            r = (delta_tscore(gold,reference,altered),config,index)
        except ZeroDivisionError: 
            print("Error at ",index,config)
        else:
            running_agg.append(r)
            
    running_agg = sorted(running_agg + worst_n + best_n,key=lambda x:x[0])
    worst_n=running_agg[:n]
    best_n=running_agg[-n:]
    del running_agg
    #print("ckecked for worst in ",config)

In [ ]:
for (diff,config,index) in worst_n+best_n:
    print(
        f"delta_tscore {round(diff,4)}, {config}@{index}\n",
        f'gold: \t {results["reference"]["gold_results"][index]} \n',
        f'ref: \t {results["reference"]["results"][index]}\n',
        f'alt: \t {results[config]["results"][index]}\n',
    )

In [ ]:
%%time
m = 10
worst_m = []
best_m = []

for config in [key for key in results.keys() if "reference" != key]:
    running_agg = []
    for index in list(results[config]["results"].keys()):
        gold = results["reference"]["gold_results"][index]
        reference = results["reference"]["results"][index]
        altered = results[config]["results"][index]
        r = ()
        try:
            r = (tquot(gold,reference,altered),config,index)
        except ZeroDivisionError: 
            #print("Error at ",index,config)
            #there are quite a lot of errors 
            r = (0,config,index)
        running_agg.append(r)
            
    running_agg = sorted(running_agg + worst_m + best_m,key=lambda x:x[0])
    worst_m=running_agg[:m]
    best_m=running_agg[-m:]
    del running_agg
    #print("ckecked for worst in ",config)

In [ ]:
for (diff,config,index) in worst_n+best_n:
    print(
        f"tquot {round(diff,4)}, {config}@{index}\n",
        f'gold: \t {results["reference"]["gold_results"][index]} \n',
        f'ref: \t {results["reference"]["results"][index]}\n',
        f'alt: \t {results[config]["results"][index]}\n',
    )

TODO: Histogram of TScores & TQuots